In [ ]:
import numpy as np
import re
import time

current_milli_time = lambda: int(round(time.time() * 1000))

puzzle_input = """<x=-10, y=-13, z=7>
<x=1, y=2, z=1>
<x=-15, y=-3, z=13>
<x=3, y=7, z=-4>"""


In [ ]:

def create_world(sss, DBG=True):
    positions=[]
    velocities=[]
    regex = re.compile(r'[\+\-]?[0-9]+')
    for line in sss.splitlines():
        if(DBG): print(line)
        coords = re.findall(r'\d+', line)
        coords = [int(k) for k in regex.findall(line)]
        
        positions.append([coords[0],coords[1],coords[2]])
        velocities.append([0,0,0])
        if(DBG): print(positions)
    return (positions, velocities)
    

def tick(positions, velocities, index, DBG=True):

    nb_moons = len(positions)

    # for each pair
    # update velocities according to gravity

    for i in range(nb_moons):
        for j in range(i,nb_moons):
            if(DBG): print(i,j)
            for dim in range(3):
                if positions[i][dim] < positions[j][dim]:
                    velocities[i][dim] += 1
                    velocities[j][dim] -= 1
                elif positions[i][dim] > positions[j][dim]:
                    velocities[i][dim] -= 1
                    velocities[j][dim] += 1
    if (DBG):
        print("after vel update")
        for j in range(len(positions)):
            print(positions[j],velocities[j])

    # for each moon
    # update positions according to velocities
    for i in range(nb_moons):
        if(DBG): print(i)
        for dim in range(3):
            positions[i][dim] += velocities[i][dim]

    if (DBG):
        for j in range(len(positions)):
            print("after pos update")
            print(positions[j],velocities[j])

    return (positions, velocities, index)

def compute_energy(positions, velocities, DBG=True):
    total_energy = 0
    nb_moons = len(positions)
    for i in range(nb_moons):
        pot_i = np.abs(positions[i][0]) + np.abs(positions[i][1]) + np.abs(positions[i][2])
        kin_i = np.abs(velocities[i][0]) + np.abs(velocities[i][1]) + np.abs(velocities[i][2])
        energy_i = pot_i*kin_i
        if(DBG): 
            print(positions[i], velocities[i], pot_i, kin_i, energy_i)

        total_energy += energy_i
        
    return total_energy



In [ ]:
def test(cc=None, expected=None):
    result = str(function(cc))
    flag = (result == expected)
    if (expected==None):
        print(str(cc) + " -> "+ str(result))
    else:
        print(str(cc) + " -> "+str(result), " -> "+ str(flag))

In [ ]:
from collections import defaultdict
import copy

def function(sss, nb_ticks, DBG=True):

    # dict of lists
    energy_to_i=defaultdict(list)
    energy_to_pv=defaultdict(list)
    
    
    cur_time = current_milli_time()
    
    (positions, velocities) = create_world(sss, DBG)    
    
    total_energy = compute_energy(positions, velocities, DBG)
    index = 0
    pv = str([positions, velocities])
    energy_to_i[total_energy].append(index)
    energy_to_pv[total_energy].append(pv)

    #print("index,total_energy,(positions, velocities)",index,total_energy,(positions, velocities))

    for index in range(1,nb_ticks):
        if (index % 100000 ==0): 
            time_now = current_milli_time()
            print("index,time",index,(time_now-cur_time)/1000)
            cur_time = time_now
        (positions, velocities, index) = tick(positions, velocities, index, DBG)
        
        total_energy = compute_energy(positions, velocities, DBG)
        #pv = copy.deepcopy([positions, velocities])
        pv = str([positions, velocities])

        if pv in energy_to_pv[total_energy]:
            print("***")
            print("index,total_energy",index,total_energy)
            print(positions,velocities)
            print("energy_to_pv[total_energy]",energy_to_pv[total_energy])
            print("energy_to_i[total_energy]",energy_to_i[total_energy])
            iii = energy_to_pv[total_energy].index(pv)
            step_old_status = energy_to_i[total_energy][iii]
            print(step_old_status,index,(index-step_old_status))
            return (index-step_old_status)


        energy_to_i[total_energy].append(index)
        energy_to_pv[total_energy].append(pv)

        #print("index,total_energy,(positions, velocities)",index,total_energy,pv)
        #print("energy_to_i",energy_to_i)
        #print("energy_to_pv",energy_to_pv)
        #if(index==10):break


    return -1


In [ ]:

test_input ="""<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>"""

out = function(test_input, 2773, False) # 179
print(out)

test_input ="""<x=-8, y=-10, z=0>
<x=5, y=5, z=10>
<x=2, y=-7, z=3>
<x=9, y=-8, z=-3>"""

out = function(test_input, 4686774924,False) # 179
print(out)


In [ ]:

%time function(puzzle_input,1000,False)

In [ ]:

def perf(sss, nb_ticks, DBG=True):
    (positions, velocities) = create_world(sss, DBG)    
    for index in range(nb_ticks):
        (positions, velocities, index) = tick(positions, velocities, index, DBG)
    return 0

%time perf(puzzle_input,500000,False)

In [ ]:
ii32 = np.iinfo(np.int32)
print(4686774924, ii32.max)